In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
%matplotlib notebook
import load_raw as Raw
import seaborn as sns
from scipy.special import expit as logit
from sklearn.model_selection import train_test_split
from scipy.special import expit as sigmoid # is more stable in case of overflows
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math


In [ ]:
df_cleaned = pd.read_csv('/data/ortho/AllPatients.csv',  sep= ';')

# x is naar rechts
# y is omhoog
# z is rotatie????

In [ ]:
# bereken symmetrie
df_cleaned['clavicula_x_dif'] = np.absolute(df_cleaned['clavicula_l_x'] - df_cleaned['clavicula_r_x'])
df_cleaned['clavicula_y_dif'] = np.absolute(df_cleaned['clavicula_l_y'] - df_cleaned['clavicula_r_y'])
df_cleaned['clavicula_z_dif'] = np.absolute(df_cleaned['clavicula_l_z'] - df_cleaned['clavicula_r_z'])

df_cleaned['scapula_x_dif'] = np.absolute(df_cleaned['scapula_l_x'] - df_cleaned['scapula_r_x'])
df_cleaned['scapula_y_dif'] = np.absolute(df_cleaned['scapula_l_y'] - df_cleaned['scapula_r_y'])
df_cleaned['scapula_z_dif'] = np.absolute(df_cleaned['scapula_l_z'] - df_cleaned['scapula_r_z'])

df_cleaned['humerus_x_dif'] = np.absolute(df_cleaned['humerus_l_x'] - df_cleaned['humerus_r_x'])
df_cleaned['humerus_y_dif'] = np.absolute(df_cleaned['humerus_l_y'] - df_cleaned['humerus_r_y'])
df_cleaned['humerus_z_dif'] = np.absolute(df_cleaned['humerus_l_z'] - df_cleaned['humerus_r_z'])

# hulp array, met alle parameters die voor de classifier gebruikt worden, je kan hier alles in doen wat je wilt
param = [ \
          'humerus_l_x', 'humerus_l_y', 'humerus_l_z', 'humerus_r_x', 'humerus_r_y', 'humerus_r_z', \
          'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z', 'clavicula_r_x', 'clavicula_r_y', 'clavicula_r_z', \
          'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z', \
          'clavicula_x_dif','clavicula_y_dif','clavicula_z_dif', \
          'scapula_x_dif','scapula_y_dif','scapula_z_dif', \
          'humerus_x_dif', 'humerus_y_dif', 'humerus_z_dif'
         ]

df_cleaned['bias'] = 1

# split oorsprong kolom in onderdelen
x,y = df_cleaned['Oorsprong'].str.split(".").str #Oordprong word vertaald naar een string en wordt gesplits op de punt
df_cleaned['cat'],df_cleaned['pat'],df_cleaned['meting'],df_cleaned['oef'] = x.str.split("_").str #4 categorieen gemaakt obv file name
df_cleaned['cat'] = [ int(x[3:]) for x in df_cleaned['cat']] #voor elk 3+ element in de kolom wordt vertaald naar een int
df_cleaned['meting'] = [ int(x[6:]) for x in df_cleaned['meting']] 
df_cleaned['oef'] = [ int(x[3:]) for x in df_cleaned['oef']] 
df_cleaned['pat'] = [ int(x[3:]) for x in df_cleaned['pat']] 
#na deze regels te hebben uitgevoerd zijn er nieuwe categorieen met ints.

df_cleaned['pat'] = df_cleaned['cat']*1000+df_cleaned['pat'] #geef elke patient een uniek nummer

#maak boolean kolom per categorie
df_cleaned['c4'] = ['Cat4' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c3'] = ['Cat3' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c2'] = ['Cat2' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c1'] = ['Cat1' in vincent for vincent in df_cleaned['Oorsprong']]


In [ ]:
for i in range(4):
    class_2b_found = str("c" + str(i+1))

    #hulpvariabele geeft aan welke categorie we willen onderscheiden

    Xcolumns = ['bias']
    Xcolumns.extend(param)

    X2 = Xcolumns

    # 
    X = df_cleaned[Xcolumns]
    y = df_cleaned[class_2b_found]


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

    # De keuze voor een classifier is "LogisticRegression", vandaar dat lr= LogisticRegression
    lr = LogisticRegression()

    # Fit het model, met de train data (80 % )
    lr.fit(X_train, y_train)

    # predict of de test data goed past bij deze LogisticRegression
    y_pred = lr.predict(X_test)

    y_pred = lr.predict(X_train)

    #theta was niet defined in dit bestand (hadden we wel nodig
    theta = lr.coef_.flatten()

    ParamCheck = pd.DataFrame()
    ParamCheck['Params'] = list(df_cleaned[Xcolumns])
    ParamCheck['Values'] = list(lr.coef_)[0]
    ParamCheck['Absolute'] = np.absolute(list(ParamCheck['Values']))
    ParamCheck = ParamCheck.sort_values(by=['Absolute'],ascending=False)


    # functie om plot van 2 van de variabelen uit de tabel df_cleaned te maken
    def scatter(param1, param2):
        cat4 = df_cleaned.where(df_cleaned['c4'])
        cat3 = df_cleaned.where(df_cleaned['c3'])
        cat2 = df_cleaned.where(df_cleaned['c2'])
        cat1 = df_cleaned.where(df_cleaned['c1'])

        plt.plot(cat3[param1], cat3[param2], '.', color='red', markersize=2)
        plt.plot(cat1[param1], cat1[param2], '.', color='cyan', markersize=2)
        plt.plot(cat2[param1], cat2[param2], '.', color='green', markersize=2)
        plt.plot(cat4[param1], cat4[param2], '.', color='blue', markersize=2)
        plt.title(param1+'/'+param2)
        plt.ylabel(param1)
        plt.xlabel(param2);

    def plot_decision_boundary(theta):
        ax = plt.gca()
        x_min, x_max = ax.get_xlim()
        y_min, y_max = ax.get_ylim()
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 1),
                             np.arange(y_min, y_max, 1))

        X = np.matrix(np.vstack([np.ones(xx.shape[0] * xx.shape[1]), xx.ravel(), yy.ravel()])).T
        boundary = logit(X * theta)
        boundary = boundary.reshape(xx.shape)

        ax.contour(xx, yy,
               boundary,
               levels=[0.5])

    # maak plotje van 2 variabelen

    p1 = 5
    p2 = 17
    scatter(param[p1-1],param[p2-1])
    selectedtheta = np.array([theta[0], theta[p1], theta[p2]])
    selectedtheta = np.reshape(selectedtheta,(3,1))
    plot_decision_boundary(selectedtheta)


    tmp = [theta[0], theta[p1], theta[p2]]

    tmptheta = np.reshape(theta, (-1, 1))
    X = np.matrix(df_cleaned[Xcolumns])


    def logit(z):
        return 1.0 / (1.0 + np.exp(-z))

    def h(X, theta):
        return logit(X * theta)

    def predict(X, theta):
        return h(X, theta) >= 0.5

    #
    # Gevectoriseerde variant om recal en precision uit te rekenen (is sneller)
    #
    x = predict(X_test, tmptheta)

    # bereken True Positives
    def TruePositives(y_pred, y_real):
        return sum(y_pred*y_real)

    # xx = np.squeeze(np.asarray(x))
    # yy = np.squeeze(np.asarray(y))
    # TP = sum(xx*yy)
    TP = TruePositives(np.squeeze(np.asarray(x)), np.squeeze(np.asarray(y)))

    # bereken False Positives
    xx = np.squeeze(np.asarray(x))
    yy = np.squeeze(np.asarray(~y))
    FP = sum(xx*yy)

    xx = np.squeeze(np.asarray(~x))
    yy = np.squeeze(np.asarray(~y))
    TN = sum(xx*yy)

    xx = np.squeeze(np.asarray(~x))
    yy = np.squeeze(np.asarray(y))
    FN = sum(xx*yy)

    # tabel printen
    tab = [["pred pos", TP, FP], ["pred neg", FN, TN]]
    print(pd.DataFrame(tab, columns=["", "pos", "neg"]))
    print()
    print("recall", TP/(TP+FN))
    print("precision", TP/(TP+FP))
    print("accuracy", (TP+TN)/(TP+TN+FP+FN))




    # kolom met voorspelling toevoegen aan data
    # predict of de test data goed past bij deze LogisticRegression
    df_cleaned['predict'] = lr.predict(X)

    ## maak tabel (df4) met per patient het aantal sampels dat true en false gelabeld wordt.

    # aantal false tellen
    df2 = df_cleaned.groupby(['pat','predict']).size().to_frame('countF').reset_index()
    df2 = df2.where(df2['predict'] == False).dropna()
    df2 = df2.set_index('pat')

    # aantal true tellen
    df3 = df_cleaned.groupby(['pat','predict']).size().to_frame('countT').reset_index()
    df3 = df3.where(df3['predict'] == True).dropna()
    df3 = df3.set_index('pat')

    # join df2 en df3
    df4 = pd.concat([df2, df3], axis=1)
    df4.drop(['predict','predict'], axis=1, inplace=True)
    df4.fillna(0, inplace=True)

    # bereken percentage
    df4['percentage'] = 100*df4['countT'] / (df4['countT'] + df4['countF'])
    df4.sort_values(by=['percentage'],ascending=False).head(30)




    if class_2b_found == "c1" :
        resultdf1 = df4
    if class_2b_found == "c2" :
        resultdf2 = df4
    if class_2b_found == "c3" :
        resultdf3 = df4
    if class_2b_found == "c4" :
        resultdf4 = df4


In [ ]:
#collumn namen renamen
resultdf1.rename(columns={ resultdf1.columns[2]: "percentagecat1" },inplace=True)
resultdf2.rename(columns={ resultdf2.columns[2]: "percentagecat2" },inplace=True)
resultdf3.rename(columns={ resultdf3.columns[2]: "percentagecat3" },inplace=True)
resultdf4.rename(columns={ resultdf4.columns[2]: "percentagecat4" },inplace=True)

#alle resultaten in 1 df joinen
resultdf5 = pd.concat([resultdf1, resultdf2, resultdf3, resultdf4], axis=1, join='inner')

def mydef(index):
    string = str(index)
    cat = string[0]
    return cat
    
#collumn 'groundtruth' toevoegen
# resultdf5['groundtruth'] = resultdf5[.index.apply(mydef)]

# # print(resultdf5['groundtruth'])
# print(resultdf5['groundtruth'])

resultdf5.reset_index(inplace=True)
resultdf5['groundtruth'] = resultdf5['pat'].apply(mydef)

resultdf5.set_index('pat', drop=True, inplace=True)

print(resultdf5.head())

In [ ]:
if(False):
    df_cleaned = pd.read_csv('all_predictions.csv', sep=';')
    param = ['Cat1', 'Cat2', 'Cat3', 'Cat4']
    df_cleaned[['Cat1', 'Cat2', 'Cat3', 'Cat4']] = df_cleaned[['Cat1', 'Cat2', 'Cat3', 'Cat4']].stack().str.replace(',','.').unstack()
    print(df_cleaned.head())
    yValueColumn = 'groundtruth'
    X_param = np.matrix(df_cleaned[param])
    y = df_cleaned[yValueColumn]
    print(df_cleaned.head())

In [ ]:
    param = ['percentagecat1', 'percentagecat2', 'percentagecat3', 'percentagecat4']
    print(param)
yValueColumn = 'groundtruth'    
X_param = np.matrix(resultdf5[param])
y = resultdf5['groundtruth']

In [ ]:
# Use SKLearn.linear_model
model_param = LinearRegression()
model_param.fit(X_param, y)

In [ ]:
y_pred = model_param.predict(X_param)
mean_squared_error(y_pred, y)

In [ ]:
# kolom met voorspelling toevoegen aan data)
resultdf5['predict'] = y_pred
resultdf5['predict_round'] = [int(round(i)) if i <4 and i > 1 else 4 if i > 4 else 1 for i in y_pred]
resultdf5 = resultdf5.drop(['countF', 'countT', ], axis=1)
resultdf5.reset_index(inplace=True)
print(resultdf5.columns)

In [ ]:
## maak tabel (df4) met per patient het aantal sampels dat true en false gelabeld wordt.

# aantal false tellen
df2 = resultdf5.groupby(['pat','predict_round']).size().to_frame('countF').reset_index()
print(df2.head())
df2.set_index(['pat', 'predict_round'], inplace=True)
print(df2.head())
df2 = df2.unstack(-1)
print(df2.head())
print('test')
print(resultdf5[['pat',yValueColumn]])
df2 = df2.join(resultdf5[['pat',yValueColumn]].drop_duplicates(subset=['pat', yValueColumn]).set_index('pat'))
print(df2.head())



In [ ]:
def CalculatePerc(cat, CountF1, CountF2, CountF3, CountF4):
    Count = [CountF1, CountF2, CountF3, CountF4]
    cat = int(cat)
    perc = 100*Count[cat-1]/np.nansum(Count)
    if(math.isnan(perc)):
        perc = 0
    return perc

df2['percentage'] = np.vectorize(CalculatePerc)(df2[yValueColumn], df2['countF', 1],df2['countF', 2],df2['countF', 3],df2['countF', 4])
df2.sort_values(by=['percentage'],ascending=False)